# 🧠 **Section VI: How to Perform Conditional Processing**

---

## 📋 **Topics to Cover**

**A. Comparison instructions create Boolean results**

**B. How to perform conditional jumps and loops**

**C. Using conditional processing to mimic conventional IF and WHILE branching**

**D. `.REPEAT` and `.WHILE` directives**


## A. Comparison Instructions Create Boolean Results

Assembly uses comparison instructions like `cmp` and `test` to set status flags based on a Boolean result.

* `cmp op1, op2` performs `op1 - op2` and sets flags (Zero Flag, Sign Flag, etc.).
* `test op1, op2` performs a bitwise AND and sets flags (commonly used to test if a value is zero or negative).

---

### ✅ Example: CMP Instruction in NASM (x86\_64 Linux)

This program compares two integers and prints whether one is greater, less, or equal.



In [ ]:
# 🛠️ Install required tools
!apt update -qq && apt install -y nasm gcc -qq

# 📝 Write the assembly program to file
asm_code = """
section .data
    msg_equal db "Equal", 0xa
    msg_gt db "Greater", 0xa
    msg_lt db "Less", 0xa

    len_equal equ $ - msg_equal
    len_gt equ $ - msg_gt
    len_lt equ $ - msg_lt

section .text
    global _start

_start:
    mov eax, 5
    mov ebx, 10
    cmp eax, ebx
    je equal
    jg greater

less:
    mov rax, 1
    mov rdi, 1
    mov rsi, msg_lt
    mov rdx, len_lt
    syscall
    jmp done

greater:
    mov rax, 1
    mov rdi, 1
    mov rsi, msg_gt
    mov rdx, len_gt
    syscall
    jmp done

equal:
    mov rax, 1
    mov rdi, 1
    mov rsi, msg_equal
    mov rdx, len_equal
    syscall

done:
    mov rax, 60
    xor rdi, rdi
    syscall
"""

with open("cmp_example.asm", "w") as f:
    f.write(asm_code)

# ⚙️ Assemble and link the program
!nasm -f elf64 cmp_example.asm -o cmp_example.o
!ld cmp_example.o -o cmp_example

# ▶️ Run the program
!./cmp_example

## B. How to Perform Conditional Jumps and Loops

In Assembly language, **conditional jumps** allow you to make decisions and create loops — just like `if`, `while`, or `for` statements in high-level languages like Python or C.

---

### 🔁 Types of Conditional Jumps (x86)

| Instruction   | Meaning                               |
| ------------- | ------------------------------------- |
| `je` / `jz`   | Jump if Equal / Zero (ZF = 1)         |
| `jne` / `jnz` | Jump if Not Equal / Not Zero (ZF = 0) |
| `jg` / `jnle` | Jump if Greater                       |
| `jl` / `jnge` | Jump if Less                          |
| `jge`         | Jump if Greater or Equal              |
| `jle`         | Jump if Less or Equal                 |

---

### 🔄 Looping with `jmp` and Labels

You can create loops by jumping back to a label. Example structure:

```asm
mov ecx, 5        ; loop 5 times
loop_start:
    ; some instructions
    loop loop_start  ; decrements ECX, jumps if ECX != 0
```

---

### ✅ Example: Countdown Loop in NASM

This example prints a countdown from 5 to 1 using a loop and conditional jump.

In [ ]:
# Install NASM and GCC
!apt update -qq && apt install -y nasm gcc -qq

# Write countdown.asm to disk
asm_code = """
section .data
    numbers db "5", 0xa, "4", 0xa, "3", 0xa, "2", 0xa, "1", 0xa
    len equ 10

section .text
    global _start

_start:
    mov rsi, numbers      ; Point to start of string
    mov rcx, 5            ; Loop counter (5 numbers)

print_loop:
    mov rax, 1            ; syscall: write
    mov rdi, 1            ; file descriptor: stdout
    mov rdx, 2            ; write 2 bytes (digit + newline)
    syscall

    add rsi, 2            ; move to next number
    loop print_loop       ; RCX -= 1, if RCX != 0, jump to print_loop

    mov rax, 60           ; syscall: exit
    xor rdi, rdi
    syscall
"""

with open("countdown.asm", "w") as f:
    f.write(asm_code)

!nasm -f elf64 countdown.asm -o countdown.o
!ld countdown.o -o countdown
!./countdown

---

### 🧠 What You Learned

* `loop` uses `rcx` (or `ecx`) as a counter. It jumps to a label if the counter is not 0.
* `jmp` is an **unconditional jump**.
* Conditional jumps like `je`, `jne`, `jg`, etc., use the results of `cmp` to decide **whether to jump**.

## C. Using Conditional Processing to Mimic IF and WHILE Branching

In high-level languages, we write control logic using constructs like:

```python
if x > 10:
    # do something
```

Or:

```python
while x > 0:
    # loop
```

In Assembly, we **mimic** these using `cmp` followed by **conditional jumps** (`je`, `jne`, `jg`, `jl`, etc.).

---

### 🧠 Mimicking an `IF` Statement

In Assembly:

```asm
cmp eax, 10     ; compare eax to 10
jle skip        ; if eax <= 10, skip the block
; Code for EAX > 10
skip:
```

This mimics:

```c
if (eax > 10) {
    // do something
}
```

---

### 🔄 Mimicking a `WHILE` Loop

```asm
mov ecx, 5         ; loop while ecx > 0
loop_start:
    cmp ecx, 0
    jle loop_end   ; if ecx <= 0, exit loop

    ; do something

    dec ecx
    jmp loop_start
loop_end:
```

This mimics:

```c
while (ecx > 0) {
    // do something
    ecx--;
}
```

---

### ✅ Example: Simulated IF + WHILE in NASM


In [ ]:
# Install assembler tools
!apt update -qq && apt install -y nasm gcc -qq

# Write simulated_if_while.asm
asm_code = """
section .data
    msg db "Inside loop!", 0xa
    len equ $ - msg

section .text
    global _start

_start:
    mov ecx, 3      ; while ecx > 0

loop_start:
    cmp ecx, 0
    jle loop_end

    ; Simulate IF: only print if ecx is not 2
    cmp ecx, 2
    je skip_print

    ; print message
    mov eax, 1
    mov edi, 1
    mov rsi, msg
    mov edx, len
    syscall

skip_print:
    dec ecx
    jmp loop_start

loop_end:
    mov eax, 60
    xor edi, edi
    syscall
"""

with open("simulated_if_while.asm", "w") as f:
    f.write(asm_code)

!nasm -f elf64 simulated_if_while.asm -o sim_if.o
!ld sim_if.o -o sim_if
!./sim_if

---

### 🧠 What You Learned

* You can simulate `if` with `cmp` and a jump over the block.
* You can simulate `while` with a label and a conditional jump back.
* Jump instructions give Assembly flexible control flow — just like structured logic in higher languages.

---

## D. `.REPEAT` and `.WHILE` Directives (MASM-style syntax)

In some Assembly environments — like **MASM (Microsoft Macro Assembler)** — you have access to **pseudo high-level directives** that resemble common loop structures.

These are not actual CPU instructions, but **macros** or **directives** that get translated into real Assembly instructions when assembling.

---

### 🧠 What Are Directives?

Directives like `.REPEAT`, `.WHILE`, and `.IF` are part of MASM/TASM syntax. They're meant to make code easier to read and write, especially for beginners.

They are **not available in NASM** (the assembler we're using in Linux/Colab).

---

### 🔄 `.REPEAT` / `.UNTIL` Example (MASM)

```asm
mov cx, 5
.REPEAT
    ; Code block
    dec cx
.UNTIL cx == 0
```

**Translation in raw assembly:**

```asm
mov cx, 5
repeat_label:
    ; Code block
    dec cx
    cmp cx, 0
    jne repeat_label
```

---

### ⏳ `.WHILE` / `.ENDW` Example (MASM)

```asm
mov cx, 5
.WHILE cx != 0
    ; Code block
    dec cx
.ENDW
```

**Translated to real assembly:**

```asm
mov cx, 5
while_start:
    cmp cx, 0
    je while_end
    ; Code block
    dec cx
    jmp while_start
while_end:
```

---

### 📌 Key Points

* These directives are shortcuts; the assembler expands them to actual `cmp`, `jmp`, and label instructions.
* NASM doesn't support these. If you're using Linux/Colab, stick to **manual label + jump** patterns.
* You may see `.IF`, `.WHILE`, `.REPEAT`, `.BREAK`, `.CONTINUE`, etc., in Windows Assembly tutorials.

---

### ✅ Summary of VI.D

| Directive            | Equivalent Instruction Pattern  |
| -------------------- | ------------------------------- |
| `.REPEAT` / `.UNTIL` | label → body → `cmp` → `jne`    |
| `.WHILE` / `.ENDW`   | `cmp` → `je` → body → jump back |


# ✅ **Summary: Section VI - How to Perform Conditional Processing**

This section taught how low-level assembly achieves **decision-making and control flow**, just like `if`, `while`, and `for` in high-level programming.

---

### 🔹 **A. Comparison Instructions Create Boolean Results**

* `cmp` subtracts two values and sets flags (Zero, Sign, etc.).
* `test` performs bitwise AND and sets flags (for checking if values are zero or negative).
* These flags are used by conditional jump instructions.

---

### 🔹 **B. Performing Conditional Jumps and Loops**

* Conditional jump instructions (`je`, `jne`, `jg`, `jl`, etc.) make the CPU jump to a different part of the program **if certain conditions are true**.
* Loops can be created by:

  * Using labels (`start:` and `jmp start`)
  * Using counters with `loop` instruction

---

### 🔹 **C. Mimicking IF and WHILE in Assembly**

* `if` blocks are created using `cmp` + conditional jumps to skip or enter code blocks.
* `while` loops are built by:

  * Label at start of loop
  * Comparison + conditional jump to end
  * Jump back to start of loop

---

### 🔹 **D. Using `.REPEAT`, `.WHILE`, and Other Directives**

* These are **pseudo-instructions/macros** available in **MASM**, not NASM.
* They mimic high-level constructs (`.WHILE`, `.IF`, `.REPEAT`) but are not real CPU instructions.
* They internally translate into labels and conditional jumps.

---

### 📌 Final Thoughts

* Conditional processing is at the heart of **program logic**.
* You now know how to write basic decision structures and loops using Assembly.
* The key tools: `cmp`, conditional jumps (`je`, `jne`, `jl`, etc.), and labels.